In [6]:
import pandas as pd
import numpy as np
from scipy.stats import norm


#WYSCOUT
df = pd.read_excel('/Users/marclambertes/Downloads/WSL GK.xlsx')
#df['Player'] = df['Player'].str.split('\\', expand=True)[0]
#df = df[df['Position'] =='LW|RW']

#df = df[df['Position'] =='CF']

df['Position'].fillna('Unknown', inplace=True)
# Filter the DataFrame based on the 'Position' column
#df = df[df['Position'].str.contains('CF')]
df = df[df['Position'].str.contains('GK')]
#df = df[df['Position'].str.contains('^(?!.*CF|.*LW|.*RW|.*AMF).*CM|.*DM$', regex=True)]
#df = df[df['Minutes played'] > 500]
#df = df[df['Age'] <= 30]

#df = df[df['Tier'] =='Tier 2']

#df = df[df['League'] == 'Belgian First Division B']
#df['League'] = df['League'].replace('2. Frauen-Bundesliga', '2. Frauen-BuLi')
df['Team within selected timeframe'] = df['Team within selected timeframe'].replace('Charleston Southern Buccaneers', 'Charleston Buccaneers')


# Classic winger
original_metrics = ["Aerial duels per 90", "Aerial duels won, %", "Save rate, %",
              "Interceptions per 90", "Progressive passes per 90", "Long passes per 90",
                   "Accurate passes, %", "Short / medium passes per 90"]

weights = [0.5, 0.5, 5, 
           0.5, 2, 0.5, 
           1, 3]  

#Playmaker winger
new_metrics = ["Aerial duels per 90", "Aerial duels won, %", "Save rate, %",
              "Interceptions per 90", "Progressive passes per 90", "Long passes per 90",
                   "Accurate passes, %", "Short / medium passes per 90"]

# Adjust the weights for the new metrics as desired
new_weights = [0.5, 0.5, 3,
              2, 2, 2,
              0.5, 5]


# Calculate the composite score for the original metrics
df["Shot-stopper"] = np.dot(df[original_metrics], weights)

# Calculate the mean and standard deviation for the composite score of the original metrics
original_mean = df["Shot-stopper"].mean()
original_std = df["Shot-stopper"].std()

# Calculate the z-scores for the composite score of the original metrics
df["Shot-stopper"] = (df["Shot-stopper"] - original_mean) / original_std

# Map the z-scores of the original metrics to a range of 0 to 100 with two decimal places
df["Shot-stopper (0-100)"] = (norm.cdf(df["Shot-stopper"]) * 100).round(2)

# Sort the DataFrame by the z-scores of the original metrics in ascending order
df_original = df.sort_values("Shot-stopper")

# Save the DataFrame with the z-scores of the original metrics to an Excel file
original_output_filename = "Shot-stopper.xlsx"
df_original.to_excel(original_output_filename, index=False)

# Calculate the composite score for the new metrics
df["Ball player"] = np.dot(df[new_metrics], new_weights)

# Calculate the mean and standard deviation for the composite score of the new metrics
new_mean = df["Ball player"].mean()
new_std = df["Ball player"].std()

# Calculate the z-scores for the composite score of the new metrics
df["Ball player"] = (df["Ball player"] - new_mean) / new_std

# Map the z-scores of the new metrics to a range of 0 to 100 with two decimal places
df["Ball player (0-100)"] = (norm.cdf(df["Ball player"]) * 100).round(2)

# Sort the DataFrame by the z-scores of the new metrics in ascending order
df_new = df.sort_values("Ball player")

# Map the z-scores of the new metrics to a range of 0 to 100 with two decimal places
df["Ball player (0-100)"] = (norm.cdf(df["Ball player"]) * 100).round(2)

# Save both sets of z-scores to the same Excel file in the same sheet
output_filename = "z_scores_combined.xlsx"
with pd.ExcelWriter(output_filename) as writer:
    df.to_excel(writer, sheet_name="Z-Scores", index=False)

# Print the list of player names, squads, competitions, and the z-scores for both sets of metrics
print("Player Z-Scores (0-100):")
print(df[["Player", "Age", "Team within selected timeframe", "Shot-stopper (0-100)", "Ball player (0-100)"]])





Player Z-Scores (0-100):
           Player  Age Team within selected timeframe  Shot-stopper (0-100)  \
0        M. Earps   30              Manchester United                 91.29   
1       M. Arnold   29                West Ham United                 52.06   
2   M. Zinsberger   27                        Arsenal                 93.75   
3         R. Laws   32                      Liverpool                 50.47   
4      E. Roebuck   23                Manchester City                 66.54   
5        M. Walsh   28         Brighton & Hove Albion                 43.73   
6      H. Hampton   22                    Aston Villa                 64.70   
7       A. Berger   32                        Chelsea                 84.14   
8      C. Brosnan   27                        Everton                 40.09   
9      J. Leitzig   24             Leicester City WFC                 72.99   
10     G. Moloney   30                        Reading                 33.67   
11     R. Spencer   32     